In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.regularizers import l2
import pickle
from google.colab import drive


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Machine_Learning_Data/classification.csv'
df = pd.read_csv(data_path)

In [5]:
print(df.head())

                                                text  emotion
0                              feel incredibly weepy        0
1                      feeling contented wife mother        1
2  pick novels feel like dropping luggage signing...        1
3  little extra interest right feeling like husba...        1
4  feel become even lot vital high eighty five we...        1


In [6]:
# Séparation des données en entrée (textes) et étiquettes (sentiments)
texts = df['text'].values  # tous les textes sont stockés dans le tableau texts
labels = df['emotion'].values  # toutes les étiquettes sont stockées dans le tableau labels

In [7]:
# Tokenisation des textes
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding des séquences
padded_sequences = pad_sequences(sequences, maxlen=150)

In [8]:
# Division des données en ensembles d'entraînement, de validation et de test
X_train, X_temp, y_train, y_temp = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Afficher la taille de chaque ensemble
print(f"Taille de l'ensemble d'entraînement: {len(X_train)}")
print(f"Taille de l'ensemble de validation: {len(X_val)}")
print(f"Taille de l'ensemble de test: {len(X_test)}")

Taille de l'ensemble d'entraînement: 22626
Taille de l'ensemble de validation: 2828
Taille de l'ensemble de test: 2829


In [9]:
#  Construction du modèle avec régularisation
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=150))  # Augmentation de la dimension de l'embedding
model.add(GRU(units=32, return_sequences=True, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))  # Réduction des unités avec régularisation L2
model.add(GRU(units=16, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(0.4))  # Augmentation du taux de dropout
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

# Compilation du modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/10
354/354 [==============================] - 24s 52ms/step - loss: 0.6993 - accuracy: 0.8148 - val_loss: 0.2859 - val_accuracy: 0.9346
Epoch 2/10
354/354 [==============================] - 8s 21ms/step - loss: 0.2022 - accuracy: 0.9638 - val_loss: 0.2381 - val_accuracy: 0.9406
Epoch 3/10
354/354 [==============================] - 7s 19ms/step - loss: 0.1468 - accuracy: 0.9799 - val_loss: 0.2017 - val_accuracy: 0.9516
Epoch 4/10
354/354 [==============================] - 7s 18ms/step - loss: 0.1245 - accuracy: 0.9855 - val_loss: 0.2048 - val_accuracy: 0.9484
Epoch 5/10
354/354 [==============================] - 6s 18ms/step - loss: 0.1129 - accuracy: 0.9889 - val_loss: 0.2189 - val_accuracy: 0.9448
Epoch 6/10
354/354 [==============================] - 5s 15ms/step - loss: 0.1037 - accuracy: 0.9912 - val_loss: 0.2085 - val_accuracy: 0.9473
Epoch 7/10
354/354 [==============================] - 7s 21ms/step - loss: 0.1000 - accuracy: 0.9906 - val_loss: 0.2198 - val_accuracy: 0.944

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

89/89 [==============================] - 1s 7ms/step - loss: 0.2167 - accuracy: 0.9509
Loss: 0.2167314887046814, Accuracy: 0.9508660435676575


In [ ]:
# Après l'entraînement
model.save('/content/drive/My Drive/Machine_Learning_Data/sentiment_analysis_model.h5')


# Sauvegarde du tokenizer
with open('/content/drive/My Drive/Machine_Learning_Data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
